In [2]:
import wfdb
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import numpy as np
import math
import numpy as np

In [3]:
fileName=['a01','a02','a03','a04','a05','a06','a07','a08','a09','a10','a11','a12','a13','a14','a15','a16','a17','a18','a19','a20',
         'b01','b02','b03','b04','b05','c01','c02','c03','c04','c05','c06','c07','c08','c09','c10',
         'x01','x02','x03','x04','x05','x06','x07','x08','x09','x10','x11','x12','x13','x14','x15','x16','x17','x18','x19','x20']
mypath='C:/Users/sara/Documents/5anno/TESI/DataBase_PhysioNet/www.physionet.org/physiobank/database/apnea-ecg/'

In [48]:
df_features_list=[]
#for index in range(0,35):
index=8
print(index)
recordname=mypath+fileName[index]
record = wfdb.rdsamp(recordname)
dfECG=pd.DataFrame()
ecg=record[0]
Fs_ecg=record[1]['fs']
intervallo=len(ecg)/(len(ecg)*Fs_ecg)
time_ecg=[]
for i in range(0,len(ecg)):
    time_ecg.append(intervallo*i)
dfECG['time']=time_ecg
dfECG['ecg']=ecg

# valori label APNEA
dfAPNEA=pd.DataFrame()
ann = wfdb.rdann(recordname, extension="apn")
time_apn=ann.sample/100
apn=ann.symbol

dfAPNEA['time']=time_apn
dfAPNEA['apnea']=apn

# valori QRS
#dfQRS=pd.DataFrame()
#ann = wfdb.rdann(recordname, extension="qrs")
#time_qrs=ann.sample/100
#dfQRS['time']=time_qrs


#leggo RR 
dfQRS = pd.read_pickle( mypath+'RR-EDR/'+fileName[index]+'RR.pkl')
dfEDR = pd.read_pickle( mypath+'RR-EDR/'+fileName[index]+'EDR.pkl')
RR_intervals=dfQRS['rr']
EDR=dfEDR['edr']
#leggo EDR
# dfEDR = pd.read_pickle( mypath+'RR-EDR/'+fileName[index]+'EDR.pkl')
#edr=dfEDR['edr']

#divido in epoch di 60 s
lunghezza_intervalli=60*Fs_ecg
df=pd.DataFrame();
startTime=[]
stopTime=[]
startRRIndex=[]
stopRRIndex=[]
#  startEDRIndex=[]
# stopEDRIndex=[]

numIntervalli=len(dfECG)/lunghezza_intervalli
if(len(ecg)%lunghezza_intervalli!=0):
    numIntervalli+=1
# print(numIntervalli)
for i in range(0,math.floor(numIntervalli)-1):
   # mediaECG.append((ecg[i*lunghezza_intervalli:i*lunghezza_intervalli+lunghezza_intervalli-1]).mean())
    startTime.append(dfECG['time'][i*lunghezza_intervalli])
    stopTime.append(dfECG['time'][i*lunghezza_intervalli+lunghezza_intervalli-1])
   # devECG.append(np.std(ecg[i*lunghezza_intervalli:i*lunghezza_intervalli+lunghezza_intervalli-1]))

df['startTime']=startTime  
df['stopTime']=stopTime  
df['time_apn']=dfAPNEA['time']
df['label']=dfAPNEA['apnea']

#------------- DIVISIONE RR IN GRUPPI DI 60 s-------------
k=0
for i in range (0, len(df)):
        if(k<len(RR_intervals)):
            startRRIndex.append(k)
            k=k+1
            while( k<len(dfQRS) and df['stopTime'][i]>=dfQRS['time'][k] ):
                k=k+1
            stopRRIndex.append(k-1)
            #print(k-1)
       # else:
          #  startRRIndex.append(np.nan)
           # stopRRIndex.append(np.nan)

 #------------- DIVISIONE EDR IN GRUPPI DI 60 s-------------
k=0
startEDRIndex=[]
stopEDRIndex=[]
for i in range (0, len(df)):  
     if(k<len(EDR)):
            startEDRIndex.append(k)
            k=k+1
            while( k<len(dfEDR) and df['stopTime'][i]>=dfEDR['time'][k] ):
                k=k+1
            stopEDRIndex.append(k-1)

df['startRRIndex']=startRRIndex
df['stopRRIndex']=stopRRIndex
df['startEDRIndex']=startEDRIndex
df['stopEDRIndex']=stopEDRIndex
df=df.dropna()
df.to_pickle(mypath+'DivisioneDB/'+fileName[index]+'divisione.pkl')

8


In [49]:
df

,startTime,stopTime,time_apn,label,startRRIndex,stopRRIndex,startEDRIndex,stopEDRIndex
0,0.0,59.99,0.0,N,0,66,0,119
1,60.0,119.99,60.0,N,67,129,120,239
2,120.0,179.99,120.0,N,130,198,240,359
3,180.0,239.99,180.0,N,199,264,360,479
4,240.0,299.99,240.0,N,265,331,480,599
5,300.0,359.99,300.0,N,332,398,600,719
6,360.0,419.99,360.0,N,399,466,720,839
7,420.0,479.99,420.0,N,467,532,840,959
8,480.0,539.99,480.0,N,533,596,960,1079
9,540.0,599.99,540.0,N,597,662,1080,1199


In [35]:
EDR_time=dfEDR['time']
EDR_time[62039]

31020.342211676503

In [39]:
dfQRS['time'][25]

295.27